# Simulation of a random protein distribution in a spherical synapse
#### (SynCAM was distributed to the surface, while MPP2 and PSD95 were distributed in the postsynaptic lumen) 

In [ ]:
import pandas as pd
import numpy as np
import math
import random

import matplotlib.pyplot as plt
%matplotlib inline  
import os
import ntpath
from tqdm import trange
import scipy.spatial

import random

# Definition of the Simulation Function
#### This function takes a list of subject and reference diameters and places each subjects with a defined number of randomly chosen references in a 3d sphere with defined diameter and calculates the NN distribution for all (or, for the sake of speed a limeted number of) subjects.


In [ ]:
# Functions to transfer Volumes into diameters and back

def VolToDiameter(volume):
    d=2*(volume*((3/4)/math.pi))**(1/3)
    return d

def DiameterToVolume(d):
    volume=((d/2)**(3))/(((3/4)/math.pi))
    return volume
#######################################################


#volsA and volsB:  volumes, diameterRegion: diameter of single regions e.g. cells or synapses, refsPerRegion: number of references per analyzed region, maxSubjects: number of maximal subjects per simulation use max to analyze all or add number, SubjOnSurface,RefOnSurface, SubjOnSurface/RefOnSurface: restricts the one or the other channel onto the surface, if Ture. 
def NN(volsSubj,volsRef,diameterRegion,refsPerRegion,maxSubjects,SubjOnSurface,RefOnSurface):
    
    centerDistances=[]
    surfaceDistances=[]
    
    volsSubj=volsSubj[volsSubj<DiameterToVolume(diameterRegion)*0.8]#reject objects that would clot the sphere
    volsRef=volsRef[volsRef<DiameterToVolume(diameterRegion)*0.8]#reject objects that would clot the sphere
        

    if maxSubjects==max:
        tmp=len(volsSubj)
    else:
        if maxSubjects<len(volsSubj):
            tmp=maxSubjects
        else:
            tmp=len(volsSubj)
    
    for i in range (tmp):
        
        #pick random volumes

        randomSubj=random.sample(list(volsSubj),1) #pick one subject
        randomRef=random.sample(list(volsRef),int(refsPerRegion)) #pick multiple references
        
        
        tmpCenter=[]
        tmpSurface=[]
                
        
        for j in range(int(refsPerRegion)):
        
        #place objects into the defined region, 0 is the center of the sphere  
            

            diameterSubj=VolToDiameter(randomSubj[0])
            diameterRef=VolToDiameter(randomRef[j])            

            radBorderSubj=(diameterRegion/2)-(diameterSubj/2) #define radius in which Subjects can be placed
            radBorderRef=(diameterRegion/2)-(diameterRef/2) #define radius in which Reference can be placed

            
            #random subject and reference center within region --> strategy: generate cube and reject ones with too high radius
            radi=99999999999999
            while (radi>radBorderSubj):
                cSubj=[-radBorderSubj+random.random()*(2*radBorderSubj),-radBorderSubj+random.random()*(2*radBorderSubj),-radBorderSubj+random.random()*(2*radBorderSubj)]
                radi=(np.sum(np.multiply(cSubj,cSubj)))**(1/2)#pytagoras from center
            if(SubjOnSurface==True):
                cSubj[2]=(((radBorderSubj**2)-(cSubj[0]**2)-(cSubj[1]**2))**0.5)*random.choice([-1, 1])
                np.random.shuffle(cSubj)

                    
            radi=99999999999999
            while (radi>radBorderRef):
                cRef=[-radBorderRef+random.random()*(2*radBorderRef),-radBorderRef+random.random()*(2*radBorderRef),-radBorderRef+random.random()*(2*radBorderRef)]
                radi=(np.sum(np.multiply(cRef,cRef)))**(1/2)#pytagoras from center
            if(RefOnSurface==True):
                cRef[2]=(((radBorderRef**2)-(cRef[0]**2)-(cRef[1]**2))**0.5)*random.choice([-1, 1])
                np.random.shuffle(cRef)


            
            #calculate distance between centers and surface
            deltaC=np.array(cSubj)-np.array(cRef)
            distanceC=(np.sum(deltaC*deltaC))**(1/2)#pytagoras from delta array
            distanceS=(distanceC-(diameterRef/2)-(diameterSubj/2))
            distanceS=np.clip(distanceS,0,distanceS)
                
            tmpCenter+=[distanceC]
            tmpSurface+=[distanceS]
            

        centerDistances+=[distanceC.min()]
        surfaceDistances+=[distanceS.min()]
                       
    NNcenter=pd.DataFrame(np.array(centerDistances))
    NNsurface=pd.DataFrame(np.array(surfaceDistances))
    table=pd.concat([NNcenter, NNsurface], axis=1)
    table=pd.DataFrame(table)
    table.columns = ['NNcenter', 'NNsurface']
    table
    
    return table    
    

# Test of the simulation on object from one image

In [ ]:
# load the list of object size
PSD=pd.read_table("..../test_simulation/Ch2shifted-8pixels_Div21_hippoN_No23_647-PSD95_568-MPP2-488_SynCAM_405-vGlut_oil18_001_visit_2_SIR_ALX.tif_PSD95_Pipeline_V3.2.txt", sep="\t")
MPP2=pd.read_table("....//test_simulation/Ch2shifted-8pixels_Div21_hippoN_No23_647-PSD95_568-MPP2-488_SynCAM_405-vGlut_oil18_001_visit_2_SIR_ALX.tif_MPP2_Pipeline_V3.2.txt", sep="\t")
SynCAM=pd.read_table("....//test_simulation/Ch2shifted-8pixels_Div21_hippoN_No23_647-PSD95_568-MPP2-488_SynCAM_405-vGlut_oil18_001_visit_2_SIR_ALX.tif_SynCAM_Pipeline_V3.2.txt", sep="\t")

PSDvolsUM=PSD["Volume: Volume"]*1000**6
SynCAMvolsUM=SynCAM["Volume: Volume"]*1000**6
MPP2volsUM=MPP2["Volume: Volume"]*1000**6


#run the simulation
significance=0.95
rounds=10
diameterRegion=0.8#µm (published postsynaptic size)
volsSubj=SynCAMvolsUM
volsRef=MPP2volsUM

refsPerRegion= len(volsRef)/len(PSDvolsUM)
tmp=NN(volsSubj,volsRef,diameterRegion,refsPerRegion,100,False,False)
#volsA and volsB:  volumes, diameterRegion: diameter of single regions e.g. cells or synapses, refsPerRegion: number of references per analyzed region, maxSubjects: number of maximal subjects per simulation use max to analyze all or add number, SubjOnSurface,RefOnSurface, SubjOnSurface/RefOnSurface: restricts the one or the other channel onto the surface, if Ture.

tmp.hist(bins=100,range=[-1,1])
plt.show()


# Defining further Functions:

## Define Data Binning Function

In [ ]:
from scipy.stats import binned_statistic

def binData(inData,firstcol,lastcol,minbin,maxbin,nbins):
    myArray=inData.as_matrix()
    
    numrows = len(myArray)
    numcols = len(myArray[0]) #[0] for cols. otherwise rows
    if (lastcol=="max"):
        lastcol=numcols

    for j in range(firstcol, lastcol):
       

        data = myArray[:,j]
        bin_means = binned_statistic(data, data, statistic="count", bins=nbins, range=(minbin, maxbin))[0]
        bin_edges= binned_statistic(data, data, statistic='count', bins=nbins, range=(minbin, maxbin))[1]
        binnumber= binned_statistic(data, data, statistic='count', bins=nbins, range=(minbin, maxbin))[2]
        bin_middle=bin_edges[0:len(bin_edges)-1]+((maxbin-minbin)/nbins/2)

        bin_means = np.nan_to_num(bin_means)
        bin_means=bin_means.tolist()       
        
        if (j==0):
            allbins = np.vstack((bin_middle, bin_means))
        else:
            allbins = np.vstack((allbins, bin_means))
        
    allbins=allbins.T    
    matrix=pd.DataFrame(allbins).set_index(0)
    matrix.columns=inData.columns[firstcol:lastcol]     

    return(matrix)

## Define Normalization Function

In [ ]:
def normalize(raw):
    normalized=(raw/raw.max())-(raw.min()/raw.max())#notmalize 1-0
    normalized=(raw/raw.mean())#normalize to sum
    #normalized=raw
    normalized=pd.DataFrame(normalized).fillna(0)
    return normalized


#NormSubjDesi_RefERGIC=(SubjDesi_RefERGIC/SubjDesi_RefERGIC.max())-(SubjDesi_RefERGIC.min()/SubjDesi_RefERGIC.max())
#pd.DataFrame(NormSubjDesi_RefERGIC).fillna(0)


# Define Splitting Function

In [ ]:
def split(matrix,identifyer,axis):
    splitted=matrix.filter(like=identifyer,axis=axis)
    return splitted

# """ BATCH PROCESSING """

## Generate file lists per condition

In [ ]:
path=r"....\_Annotation_And_Simulations\noShift_and_simulations"

files=[]
tmp=os.listdir(path)
for each in tmp:
    if (".txt" in each)==True: 
        files=files+[each]

In [ ]:
conditions=["_PSD95_Pipe","_MPP2_Pipe","_SynCAM_Pipe"]

#create empty matrix
paths=[path +"/"+ s for s in files]
filematrix= [[]*5]*len(conditions)

#fill matrix
for i in range(0,len(conditions)):
    tmp=conditions[i]
    filematrix[i]= [x for x in paths if tmp in x]
    
filematrix=pd.DataFrame(filematrix).T
filematrix.columns=conditions

# Run the simulation from per file multiple times

In [ ]:
from IPython.display import display, clear_output

nSimulations=10
maxSubjects=10000#maximum number of subjects per simulation --> to minimize time. for best quality chose max
#maxSubjects=2

level=0.95#CI significance level
pixelsize=0.04

allNNs=pd.DataFrame()
for i in range (nSimulations):

    tmpNNs=pd.DataFrame()
    #loop through all files
    for k in trange (len(filematrix.index), desc="Images",position=0):
        try:
            name=filematrix["_PSD95_Pipe"][k]
            print(name)
            PSD=pd.read_table(name, sep="\t")
            MPP2=pd.read_table(name.replace("_PSD95_Pipe", "_MPP2_Pipe"), sep="\t")
            SynCAM=pd.read_table(name.replace("_PSD95_Pipe", "_SynCAM_Pipe"), sep="\t")

            #create masks for objects within synapsesize and PSD in the center
            psd=PSD[["Center of Mass (Ch 1): X (px)","Center of Mass (Ch 1): Y (px)"]].values*pixelsize
            refMPP2=MPP2[["Center of Mass (Ch 2): X (px)","Center of Mass (Ch 2): Y (px)"]].values*pixelsize
            refSynCAM=SynCAM[["Center of Mass (Ch 3): X (px)","Center of Mass (Ch 3): Y (px)"]].values*pixelsize
            maskPSD_MPP2=np.min(scipy.spatial.distance_matrix(psd,refMPP2),axis=0)<diameterRegion/2 #mask for objects within synapse size
            maskPSD_SynCAM=np.min(scipy.spatial.distance_matrix(psd,refSynCAM),axis=0)<diameterRegion/2 #mask for objects within synapse size

            #select volumes within synapse
            PSDvolsUM=PSD["Volume: Volume"]*1000**6
            SynCAMvolsUM=(SynCAM["Volume: Volume"]*1000**6)[maskPSD_SynCAM]
            MPP2volsUM=(MPP2["Volume: Volume"]*1000**6  )[maskPSD_MPP2]


            a=(NN(PSDvolsUM,SynCAMvolsUM,diameterRegion,(len(SynCAMvolsUM)/len(PSDvolsUM)),maxSubjects,False,True))
            a=a.add_suffix("_Sub_PSD_Ref_SynCAM")

            b=(NN(PSDvolsUM,MPP2volsUM,diameterRegion,int(len(MPP2volsUM)/len(PSDvolsUM)),maxSubjects,False,False))
            b=b.add_suffix("_Sub_PSD_Ref_MPP2")

            c=(NN(MPP2volsUM,SynCAMvolsUM,diameterRegion,int(len(SynCAMvolsUM)/len(PSDvolsUM)),maxSubjects,False,True))
            c=c.add_suffix("_Sub_MPP2_Ref_SynCAM")    

            d=(NN(SynCAMvolsUM,PSDvolsUM,diameterRegion,1,maxSubjects,True,False))
            d=d.add_suffix("_Sub_SynCAM_Ref_PSD")

            e=(NN(MPP2volsUM,PSDvolsUM,diameterRegion,1,maxSubjects,False,False))
            e=e.add_suffix("_Sub_MPP2_Ref_PSD")

            f=(NN(SynCAMvolsUM,MPP2volsUM,diameterRegion,int(len(MPP2volsUM)/len(PSDvolsUM)),maxSubjects,True,False))   
            f=f.add_suffix("_Sub_SynCAM_Ref_MPP2")

            g=(NN(SynCAMvolsUM,SynCAMvolsUM,diameterRegion,int(len(SynCAMvolsUM)/len(PSDvolsUM)),maxSubjects,True,True))   
            g=g.add_suffix("_Sub_SynCAM_Ref_SynCAM")

            h=(NN(MPP2volsUM,MPP2volsUM,diameterRegion,int(len(MPP2volsUM)/len(PSDvolsUM)),maxSubjects,False,False))   
            h=h.add_suffix("_Sub_MPP2_Ref_MPP2")


            NNs=  pd.concat([a,b,c,d,e,f,g,h], axis=1)

            base=os.path.basename(filematrix["_PSD95_Pipe"][k])
            base[0:base.find(".tif")]
            NNs=NNs.add_suffix(("_"+base))
            tmpNNs=pd.concat([tmpNNs,NNs], axis=1)

            clear_output(wait=True)
            print("Simulation "+str(i)+" of "+str(nSimulations))
        except Exception:
            pass
        
    tmpNNs=tmpNNs.add_suffix(("_Sim"+str(i)))
    allNNs=pd.concat([allNNs,tmpNNs], axis=1)

### export raw data

In [ ]:
newPath=path+"/00simulations_raw"
if (os.path.exists(newPath)==False):
    os.makedirs(newPath)

allNNs.to_csv(newPath+"/allSimulations_raw.txt", sep="\t")

# Bin and normalize the data

In [ ]:
binned=binData(allNNs,0,"max",0,1.6,20)
newPath=path+"/02_bin"
if (os.path.exists(newPath)==False):
    os.makedirs(newPath)
binned.to_csv(path+"/02_bin/02bin.txt")
    
norm_bin=normalize(binned)
newPath=path+"/03norm_bin"
if (os.path.exists(newPath)==False):
    os.makedirs(newPath)
norm_bin.index=binned.index.values
norm_bin.to_csv(path+"/03norm_bin/03norm_bin.txt")

# Do statistics and calculate the CI

In [ ]:
#calculate number of simulation rounds
simulations=[]
for i in allNNs.columns:
    simulations = simulations + [i[i.find("Sim"):]]
simulations=list(set(simulations))


#define keywords to search for and assign/group conditions
identA=["NNcenter_Sub_PSD_Ref_SynCAM",
        "NNcenter_Sub_PSD_Ref_MPP2",
        "NNcenter_Sub_MPP2_Ref_SynCAM",
        "NNcenter_Sub_SynCAM_Ref_PSD",
        "NNcenter_Sub_MPP2_Ref_PSD",
        "NNcenter_Sub_SynCAM_Ref_MPP2",
        "NNcenter_Sub_SynCAM_Ref_SynCAM",
        "NNcenter_Sub_MPP2_Ref_MPP2",
        
        "NNsurface_Sub_PSD_Ref_SynCAM",
        "NNsurface_Sub_PSD_Ref_MPP2",
        "NNsurface_Sub_MPP2_Ref_SynCAM",
        "NNsurface_Sub_SynCAM_Ref_PSD",
        "NNsurface_Sub_MPP2_Ref_PSD",
        "NNsurface_Sub_SynCAM_Ref_MPP2",
        "NNsurface_Sub_SynCAM_Ref_SynCAM",
        "NNsurface_Sub_MPP2_Ref_MPP2",]
        
#conditions to average
identB=["No23","No_3","_01-DIV28_"]#categories to average


# do statistics on groups
for i in range (0, len(identA)):     
    for h in range(0,len(simulations)):         
        for j in range (0, len(identB)):
            
            #split conditions
            split_norm_bin=split(split(split(norm_bin,simulations[h],1),identA[i],1),identB[j],1)
            split_norm_bin.index=binned.index.values
            newPath=path+"/04_split_norm_bin"
            if (os.path.exists(newPath)==False):
                os.makedirs(newPath)
            split_norm_bin.to_csv(newPath+"/04"+identA[i]+identB[j]+simulations[h]+"_norm_bin.txt")
                        
            if (j==0):
                summary = np.array(split_norm_bin.mean(axis=1))
                header = [identA[i]+identB[j]+simulations[h]]
            else:
                summary = np.vstack((summary , split_norm_bin.mean(axis=1)))
                header = header+[identA[i]+identB[j]+simulations[h]]
                
        #statistics on averages        
        stats=np.vstack((
                        pd.DataFrame(summary).T.mean(axis=1),
                        pd.DataFrame(summary).T.median(axis=1),
                        pd.DataFrame(summary).T.std(axis=1),
                        pd.DataFrame(summary).T.sem(axis=1),
                        summary))
        header=["mean"]+["median"]+["std"]+["sem"]+header
        
        stats=pd.DataFrame(stats.T, columns=header)
        stats.index=binned.index.values
        newPath=path+"/05_stats_split_norm_bin"
        if (os.path.exists(newPath)==False):
                os.makedirs(newPath)
        stats.to_csv(newPath+"/05"+identA[i]+"statistics_average_norm_bin_"+simulations[h]+".txt")
        
        
        #statistics on normalized average

        summary=normalize(summary)
        nstats=np.vstack((
                        pd.DataFrame(summary).T.mean(axis=1),
                        pd.DataFrame(summary).T.median(axis=1),
                        pd.DataFrame(summary).T.std(axis=1),
                        pd.DataFrame(summary).T.sem(axis=1),
                        summary))
         
        
        nstats=pd.DataFrame(nstats.T, columns=header)
        nstats.index=binned.index.values
        newPath=path+"/06_stats_on_stats"
        if (os.path.exists(newPath)==False):
                os.makedirs(newPath)
        nstats.to_csv(newPath+"/06"+"statistics_norm_average_norm_bin_"+simulations[h]+identA[i]+".txt") 
       
    
    #calculate stastistics over simulations

        if (h==0):
            simStats=pd.DataFrame(nstats["mean"])
        else:
            simStats=pd.concat([simStats,nstats["mean"]],axis=1)
    
    simStats.columns=simulations
    simStats=simStats.add_suffix("_"+identA[i])
    
    CIsUP=simStats.quantile(q=(1-(1-level)/2), axis=1,interpolation='midpoint')
    CIsLOW=simStats.quantile(q=((1-level)/2), axis=1,interpolation='midpoint')
    
    simStats=pd.concat([
                        CIsUP,
                        CIsLOW,
                        simStats.T.mean(),
                        simStats.T.median(),
                        simStats.T.std(),
                        simStats.T.sem(),
                        simStats
                        ],axis=1)
    simStats.columns=["upper_CI"]+["lower_CI"]+["mean"]+["median"]+["std"]+["sem"]+simulations
    newPath=path+"/07_CI_from_"+str(len(simulations))+"simulations"
    if (os.path.exists(newPath)==False):
            os.makedirs(newPath)
    simStats.to_csv(newPath+"/07_CI_from_"+str(len(simulations))+"simulations"+identA[i]+".txt") 
    

    fig=simStats[["upper_CI","lower_CI","mean"]].plot()
    #fig.suptitle(identA[i], fontsize=14, fontweight='bold')
    fig.set_title(identA[i], fontsize=14, fontweight='bold')
    plt.xlabel("NN distance (µm)")
    plt.ylabel("Normalized Frequency")                       
    plt.axis([0, 2, 0, nstats["mean"].max()])
    plt.legend(bbox_to_anchor=(1, 1), loc=1, borderaxespad=0.)
    plt.savefig(path+"/08_"+identA[i]+"simulated_kNN.png")
     

# Plot the CIs from the simulation against the real data

In [ ]:
identA_data=[
"Inverse_C_Sch3-Rch1",    
"Inverse_C_Sch2-Rch1",
"Inverse_C_Sch3-Rch2",    
"x_C_Sch3-Rch1",   
"x_C_Sch2-Rch1",    
"x_C_Sch3-Rch2",
"x_C_Sch3-Rch3",
"x_C_Sch2-Rch2",
"Inverse_S_Sch3-Rch1",    
"Inverse_S_Sch2-Rch1",
"Inverse_S_Sch3-Rch2",    
"x_S_Sch3-Rch1",   
"x_S_Sch2-Rch1",    
"x_S_Sch3-Rch2",
"x_S_Sch3-Rch3",
"x_S_Sch2-Rch2"
]

##### pair the files

In [ ]:
simulationfolder=r"....\_Annotation_And_Simulations\noShift_and_simulations\07_CI_from_10simulations"+"\\"
oneSimulationfolder=r"....\_Annotation_And_Simulations\noShift_and_simulations\06_stats_on_stats"+"\\"
datafolder=r"....\Session1-3\_NN\NEWNEW\txt\06_stats_on_stats"+"\\"

simulationfiles=[simulationfolder + s for s in os.listdir(simulationfolder)]
oneSimulationfiles=[oneSimulationfolder + s for s in os.listdir(oneSimulationfolder)]
datafiles=[datafolder + s for s in os.listdir(datafolder)]


data=[]
simulations=[]
oneSimulations=[]


#fill matrix
for i in range(0,len(identA_data)):
    tmp=identA_data[i]
    #data[i]= [x for x in datafiles if tmp in x]
    data= data+[x for x in datafiles if tmp in x and "Noshift" in x]
    
for i in range(0,len(identA)):
    tmp=identA[i]
    simulations=simulations+ [x for x in simulationfiles if tmp in x]
    oneSimulations=oneSimulations+ [x for x in oneSimulationfiles if tmp in x and "Sim0" in x]
    
filematrix=pd.DataFrame([data,simulations,oneSimulations])
filematrix.columns=identA
filematrix.index=["data","simulations","oneSimulations"]

#### plot the data

In [ ]:
for i in range (0, len(identA)):

    thisData=pd.read_csv(filematrix[identA[i]]["data"], sep=",")
    thisSimulation=pd.read_csv(filematrix[identA[i]]["simulations"], sep=",")
    thisPlot=pd.concat([thisData["mean"],thisSimulation[["upper_CI","lower_CI"]]], axis=1)
    thisPlot.index=thisData.index=thisSimulation.index=thisData['Unnamed: 0']

    dummy = pd.DataFrame(index=thisData.index, columns=["dummy"])
    dummy.fillna("NaN") 
    fig=thisPlot[["upper_CI","lower_CI","mean"]].plot(yerr = [dummy["dummy"],dummy["dummy"],thisData["sem"]])
    #fig=thisPlot[["upper_CI","lower_CI","mean"]].plot()
    
    fig.set_title(identA[i]+"_"+identA_data[i], fontsize=14, fontweight='bold')
    plt.xlabel("NN distance (µm)")
    plt.ylabel("Normalized Frequency")                       
    plt.axis([0, 1.6, 0, max(thisPlot.max())*1.2])
    plt.legend(bbox_to_anchor=(1, 1), loc=1, borderaxespad=0.)
    plt.savefig(path+"/09_"+identA[i]+"simulated_vs_raw_kNN.png", dpi=1000)
    #thisPlot.to_excel(path+"/09_"+identA[i]+"simulated_vs_raw_kNN.xls")
    
    export=pd.concat([thisData["mean"],thisData["sem"],thisSimulation[["upper_CI","lower_CI"]]], axis=1)
    export.columns=["mean data","sem data","upper_CI","lower_CI"]
    export.to_excel(path+"/09_"+identA[i]+"simulated_vs_raw_kNN.xls")

### Plot only statistics on Simulation 0  - this can be combines with bootstrap method for the supplement

In [ ]:
for i in range (0, len(identA)):

    thisData=pd.read_csv(filematrix[identA[i]]["data"], sep=",")
    thisSimulation=pd.read_csv(filematrix[identA[i]]["oneSimulations"], sep=",")
    thisPlot=pd.concat([thisData["mean"],thisSimulation["mean"]], axis=1)
    thisPlot.columns=["data_mean","simulation_mean"]
    thisPlot.index=thisData.index=thisSimulation.index=thisData['Unnamed: 0']

    fig=thisPlot[["data_mean","simulation_mean"]].plot(yerr = [thisData["sem"],thisSimulation["sem"]])    
    fig.set_title(identA[i]+"_"+identA_data[i], fontsize=14, fontweight='bold')
    plt.xlabel("NN distance (µm)")
    plt.ylabel("Normalized Frequency")                       
    plt.axis([0, 1.6, 0, max(thisPlot.max())*1.2])
    plt.legend(bbox_to_anchor=(1, 1), loc=1, borderaxespad=0.)
    plt.savefig(path+"/10"+identA[i]+"simulated_sem_vs_raw_kNN.png", dpi=1000)
    export=pd.concat([thisData["mean"],thisData["sem"],thisSimulation["mean"],thisSimulation["sem"]], axis=1)
    export.columns=["mean data","sem data","mean simulation","sem simulation"]
    export.to_excel(path+"/10"+identA[i]+"simulated_sem_vs_raw_kNN.xls")
      
    
    
    
    fig=plt.figure()
    x=thisPlot.index.values
    y=np.divide(np.subtract(thisPlot["data_mean"].values,thisPlot["simulation_mean"].values),thisPlot["simulation_mean"].values)
    plt.plot(x[0:10],y[0:10])
    plt.title(identA[i]+"_"+identA_data[i], fontsize=14, fontweight='bold')
    plt.xlabel("NN distance (µm)")
    plt.ylabel("(Normalized Frequency -Random) / Random")                       
    plt.legend(bbox_to_anchor=(1, 1), loc=1, borderaxespad=0.)
    plt.savefig(path+"/11"+identA[i]+"simulated_sem_vs_accumulation_kNN.png", dpi=1000)